#### Install Watson Python SDK

In [ ]:
!pip install --upgrade ibm-watson

In [ ]:
import json
from ibm_watson import DiscoveryV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

#### Insert the Watson Discovery API key

In [ ]:
authenticator = IAMAuthenticator('INSERT WATSON DISCOVERY API KEY')

discovery = DiscoveryV1(
    version='2019-04-30',
    authenticator=authenticator)
discovery.set_service_url('https://gateway.watsonplatform.net/discovery/api')

#### Display Watson Discovery Environment ID

In [ ]:
environments = discovery.list_environments().get_result()
print(json.dumps(environments, indent=2))

#### Copy the environment id from the above output 

In [ ]:
# pLease copy the value of the environment id from the above output below 

covid_environment_id = 'Copy Watson Discovery Environment ID Value' 


In [ ]:
collections = discovery.list_collections(covid_environment_id).get_result()
collections = [x for x in collections['collections']]
print(json.dumps(collections, indent=2))

#### Query the collection - no filter applied

In [ ]:
query_results = discovery.query(
    covid_environment_id,
    collections[0]['collection_id'],
    count=364).get_result()
#    filter='extracted_metadata.sha1::f5*',
#   return_fields='extracted_metadata.sha1').get_result()
print(json.dumps(query_results, indent=2))

#### Calculate the number of posts by city 

In [ ]:
Counts = {"chicago":0.0,"Seattle":0.0,"LosAngeles":0.0,"washingtondc":0.0,"newyork":0.0}

for result in query_results["results"]:
    try: 
       Counts[result["subreddit"]]+=1
    except KeyError:
       Counts["newyork"] += 1 
       result["subreddit"] = "newyork"
    except:
       print("Exception Occurred")
     
print (Counts)

#### Calculate the number of posts by city and entity type 

In [ ]:
Entities = {"chicago":{},"Seattle":{}, "LosAngeles":{}, "washingtondc":{},"newyork":{}}
Mentions = {"chicago":0, "Seattle":0,"LosAngeles":0, "washingtondc":0, "newyork":0}

for result in query_results["results"]:
    try:
       i = result["enriched_text"]["entities"]
       for entity in i:
          type = entity["type"]
             # print (type)
          try:
             Entities[result["subreddit"]][type] += 1
             Mentions[result["subreddit"]] += 1
             
          except KeyError:
             Entities[result["subreddit"]][type] = 1
             Mentions[result["subreddit"]] += 1
                             
          except:
             print ("Exception Occurred")
    except KeyError:
        print ("entity key error")
    except:
        print ("Other error")

print (Entities)
                             
print (Mentions)
            
    

 
#### Compute weighted sum of Insurance/Employment vulnerability by city 

In [ ]:
# Status_Factors - provides a weighting factor for each of the employment/insurance entity types
Status_Factors = {"Unemployed":0.5,"No_Health_Insurance":0.5,"Medically_insured":-0.5, "Full_Time_Employment":-0.5}

# Status - provides a weighted sum by city of the Status_Values for each entity type multiplied by the posts for that entity type. 
Status = {"chicago":0.0,"Seattle":0.0,"LosAngeles":0.0,"washingtondc":0.0,"newyork":0.0}

for entity in Entities.keys():
    for status in Status_Factors.keys():
       try:
#          print(entity,status,Entities[entity][status])
          Status[entity] += Status_Factors[status]*Entities[entity][status]
       except KeyError:
          print (entity,status)
print()            
print (Status) 

#### Compute weighted sum of Age/Health/Education vulnerability by city 

In [ ]:
Household_Factors = {"Senior_Citizen":.35, "Minor": 0.25, "Disabled":.20, "Single_Parent":.15, "No_High_School_Diploma":0.05, "High_School_Student":-.2, "University_Student":-.3}
Household = {"chicago":0, "Seattle":0, "LosAngeles":0,"washingtondc":0,"newyork":0 }

for entity in Entities.keys():
    for status in Household_Factors.keys():
       try:
          print(entity,status,Entities[entity][status])
          Household[entity] += Household_Factors[status]*Entities[entity][status]
       except KeyError:
          print (entity,status)
            
print()
print (Household)



In [ ]:
Minority_Factors =  {"Minority":.6, "ESL_Speaker":.4} 
Minority = {"chicago":0.0,"Seattle":0.0,"LosAngeles":0.0,"washingtondc":0.0,"newyork":0.0}

for entity in Entities.keys():
    for status in Minority_Factors.keys():
       try:
          print(entity,status,Entities[entity][status])
          Minority[entity] += Minority_Factors[status]*Entities[entity][status]
       except KeyError:
          print (entity,status)
            
print()            
print (Minority)


In [ ]:
House_Factors = {"Mobile_Home_Owner": 0.5, "Crowded_Living": 0.3, "No_Vehicle":0.2, "Apartment_Renter":-.25,"Home_Owner":-.65, "Car_Owner":-.1}
House = {"chicago":0.0,"Seattle":0.0,"LosAngeles":0.0,"washingtondc":0.0,"newyork":0.0}

for entity in Entities.keys():
    for status in House_Factors.keys():
       try:
          print(entity,status,Entities[entity][status])
          House[entity] += House_Factors[status]*Entities[entity][status]
       except KeyError:
          print (entity,status)
            
print()           

print (House)


#### Compute overall notional Social Vulnerabilty Index

In [ ]:
SVI = {"chicago":0.0,"Seattle":0.0,"LosAngeles":0.0,"washingtondc":0.0,"newyork":0.0}

for city in SVI.keys():
    SVI[city] = (SVI[city] + (Status[city]*.4 + Household[city]*.3 + Minority[city]*.2 + House[city]*.3))*100/Mentions[city]

 
print (SVI)